In [1]:
#@Imports

#############################
# region: Imports and Setup #
#############################
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload
#############################
# endregion                 #
#############################

import datetime
from datetime import datetime as dt
from datetime import timedelta as td

from concordia.associative_memory import importance_function
from concordia.components import agent as components

from examples.custom_components.setup import model_setup, measurement_setup
from examples.tpb import utils
from examples.tpb import components as tpb_components
from examples.tpb import memory as tpb_memory
from examples.tpb import plan as tpb_plan
from examples.tpb import agent as tpb_agent

In [2]:
SETUP_TIME=dt(year=2024,month=10,day=1,hour=18)
TIME_STEP=td(minutes=5)

model, embedder = model_setup("llama3")
measurement, clock = measurement_setup(setup_time=SETUP_TIME,time_step=TIME_STEP)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

/Users/rgelpi/.pyenv/versions/3.12.2/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/rgelpi/.pyenv/versions/3.12.2/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
config = tpb_agent.AgentConfig(
  name="Alice",
  gender="female",
  context="Alice loves to cook, but doesn't know what a Space Shuttle is.",
  goal="Alice wants to become a patissier.",
  traits="Determined, scatterbrained, picky",
  date_of_birth=dt(year=1993,month=11,day=1,hour=00),
  extras={
    'model': model,
    'embedder': embedder,  
    'importance': importance_model.importance,
    'clock_now': clock.now
    }
)

/Users/rgelpi/.pyenv/versions/3.12.2/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


As a young girl, Alice was determined to master every recipe she came across in her mother's old cookbook. She'd spend hours in the kitchen, whipping up a storm of sugar and spice, much to the delight of her family and friends. Her scatterbrained nature often led to flour-covered countertops and pots left on the stove, but her picky palate ensured that each dish was a masterpiece. As she grew older, Alice's love for cooking only intensified, and she began to dream of opening up her own bakery.

As an adult, Alice worked as a pastry chef in a small town, where she spent her days whipping up croissants and cakes for the local café. Her typical day was a whirlwind of mixing and measuring, with a dash of creativity thrown in for good measure. Despite the chaos that often ensued in her kitchen, Alice's determination to be the best drove her to experiment with new recipes and techniques. She longed to one day open up her own bakery, where she could share her passion with the community. What 

In [4]:
params = {
  "model": model,
  "config": config,
  "num_memories": 100,
  "clock_now": clock.now,
  "verbose": True,
  "num_behavs": 5,
  "num_people": 5,
  "threading": False
}

# Memory
memory = tpb_memory.EpisodicMemory(**params)

# Situation
situation = components.situation_perception.SituationPerception(
  "situation",
  model=model,
  memory=config.memory,
  agent_name=config.name,
  clock_now = clock.now,
  num_memories_to_retrieve=100,
  verbose=True
)

# TPB pipeline components
behav = tpb_components.Behaviour("behaviour", **params)
attitude = tpb_components.Attitude("attitude", components=[behav], **params)
people = tpb_components.People("people", components=[behav], **params)
motiv = tpb_components.Motivation("motivation", components=[people], **params)
norm = tpb_components.SubjectiveNorm("norm", components=[motiv], **params)
control = tpb_components.BehaviouralControl("control", components=[behav], **params)
thin_goal = tpb_components.ThinGoal("thin_goal", **params)
intention = tpb_components.BehaviouralIntention(
  "intention", 
  components=[attitude, norm, control, thin_goal], 
  **params
)
plan = tpb_plan.TPBPlan(
  "plan",
  goal = intention,
  components=[situation],
  **params
)

full_model = tpb_components.TPBModel(
  "full_model",
  components=[
    memory,
    behav,
    attitude,
    people,
    motiv,
    norm,
    control,
    thin_goal,
    situation,
    intention,
    plan
  ]
)

In [5]:
full_model.update()

[LOG] [Component] behaviour initialized.
[LOG] [Component] attitude initialized.
[LOG] [Component] people initialized.
[LOG] [Component] motivation initialized.
[LOG] [Component] control initialized.
[LOG] [Component] intention initialized.
[LOG] [Component] thin_goal initialized.
As Alice deliberated, she considered the behaviors of being late, canceling plans, and prioritizing work over social life, which could lead to strained relationships with friends and family, decreased job satisfaction, and potential long-term consequences for her mental and physical health.working as a pastry chef at a small café, and she's 38 years old (born in 1986).Memories of Alice:
[23 Jul 1986 00:00:00] When Alice was 6 years old, she spent her summer vacation helping her mother in the kitchen, learning the basics of baking and cooking. She loved measuring out ingredients and mixing batter, feeling like a real chef as she whipped up a storm of sugar and spice. Her mother's patience and guidance instille

In [6]:
full_model.update()

Based on Alice's personality and goals, here are five potential behaviors she could exhibit:

1. **Dive into research**: Given her interest in becoming a patissier, Alice might decide to spend some time researching the latest trends and techniques in pastry-making, perhaps even looking up recipes and tutorials online or in cookbooks.

2. **Practice and experiment**: With her scatterbrained nature, Alice might be tempted to jump right into trying out new recipes and experimenting with different ingredients and techniques, hoping to stumble upon something amazing.

3. **Seek guidance from others**: As someone who values determination and is willing to learn from mistakes, Alice might decide to reach out to experienced patissiers or chefs for advice and guidance on how to improve her skills and achieve her goals.

4. **Take a cooking class**: To gain some structure and formal training, Alice could consider taking a cooking class or workshop focused on pastry-making, which would allow her 

In [7]:
%autoreload 2
from examples.tpb.utils import pprint

full_model.state() >> pprint
intention.state() >> pprint

Here is Alice's revised plan for the rest of the day:

[18:00 - 19:00] Finish up any remaining tasks at the café, then head home to relax and unwind after a long day of work.

[19:00 - 20:30] Take some time to research different schools, programs, and opportunities that could help her achieve her goal of becoming a patissier. Make a list of pros and cons for each option to consider.

[20:30 - 21:30] Continue researching and taking notes on the various options she's considering.

[21:30 - 22:00] Take a short break to grab a snack or stretch her legs before diving back into research mode.

[22:00 - 23:00] Review what she's learned so far and start making a plan for how she'll pursue each option. Make a list of next steps and potential timelines.

[23:00 - 00:00] End the night by planning out her menu for the next day's café shift, taking into account any new ideas or inspiration she may have gained from her research.
After considering her options, Alice's current goal is to successfully 